#1단계: 환경 설정 및 라이브러리 불러오기

In [1]:
# 1단계: 환경 설정 및 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer
from sklearn.utils.class_weight import compute_sample_weight
from collections import Counter
import xgboost as xgb

classification_data_path = "./sample_data/Loan_Default.csv"

# 2단계: 데이터 로드 및 전처리

In [2]:
class BinomialClassificationDataLoader:
    def __init__(self, path):
        self.data = pd.read_csv(path)  # CSV 파일 로드
        self.label_column = 'Status'  # 타겟 컬럼명 지정
        self.features = None
        self.labels = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.scaler = None  # 스케일러 객체 저장용

    def preprocess(self):
        df = self.data.copy()  # 원본 데이터 복사본 생성

        # 1) 결측치 처리 방법 비교 (mean, median, mode) 결과를 토대로 mean으로 처리
        for col in df.select_dtypes(include=np.number).columns:
            df[col] = df[col].fillna(df[col].mean())  # 수치형 컬럼은 평균으로 결측치 대체

        # 2) 모델에 의미 없는 컬럼 제거 (Loan_ID, Customer_ID, Overdue_Days)
        df = df.drop(columns=['Loan_ID', 'Customer_ID', 'Overdue_Days'], errors='ignore')

        # 3) 범주형 변수 원-핫 인코딩 (ID 컬럼 제외)
        df = pd.get_dummies(df.drop(['ID'], axis=1))

        # 4) 불균형 데이터 해소를 위해 Under Sampling 적용
        label_counts = df[self.label_column].value_counts()
        min_class = label_counts.idxmin()  # 소수 클래스 라벨
        min_count = label_counts.min()  # 소수 클래스 데이터 수

        balanced_parts = []
        for label in label_counts.index:
            subset = df[df[self.label_column] == label]
            if label == min_class:
                balanced_parts.append(subset)  # 소수 클래스는 모두 사용
            else:
                # 다수 클래스는 소수 클래스 수만큼 랜덤 샘플링 (Under Sampling)
                balanced_parts.append(subset.sample(n=min_count, random_state=42))
        balanced_df = pd.concat(balanced_parts).sample(frac=1, random_state=42).reset_index(drop=True)  # 섞기

        # 5) 스케일링 방법 비교 (StandardScaler vs MinMaxScaler)
        X = balanced_df.drop(self.label_column, axis=1).values
        y = balanced_df[self.label_column].values.astype(int)
        X_train_raw, X_test_raw, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        scaler_std = StandardScaler()  # 평균 0, 표준편차 1로 변환
        scaler_minmax = MinMaxScaler()  # 0~1 사이로 변환

        X_train_std = scaler_std.fit_transform(X_train_raw)
        X_test_std = scaler_std.transform(X_test_raw)

        X_train_minmax = scaler_minmax.fit_transform(X_train_raw)
        X_test_minmax = scaler_minmax.transform(X_test_raw)

        # Logistic Regression 모델로 스케일러별 성능 비교
        tmp_model = LogisticRegression(penalty=None, class_weight='balanced', max_iter=1000, random_state=42)

        tmp_model.fit(X_train_std, y_train)
        y_pred_std = tmp_model.predict(X_test_std)
        f1_std = f1_score(y_test, y_pred_std)

        tmp_model.fit(X_train_minmax, y_train)
        y_pred_minmax = tmp_model.predict(X_test_minmax)
        f1_minmax = f1_score(y_test, y_pred_minmax)

        # print("StandardScaler F1-score:", f1_std)
        # print("MinMaxScaler F1-score:", f1_minmax)

        # 성능이 좋은 StandardScaler 선택 후 최종 적용
        self.scaler = scaler_std

        # Stratified Split 다시 수행 + 선택한 스케일러 적용
        X_train, X_test, self.y_train, self.y_test = train_test_split(
            balanced_df.drop(self.label_column, axis=1).values,
            balanced_df[self.label_column].values.astype(int),
            test_size=0.2,
            random_state=42,
            stratify=balanced_df[self.label_column]
        )
        self.X_train = self.scaler.fit_transform(X_train)
        self.X_test = self.scaler.transform(X_test)

        # 데이터 형태 및 분포 출력
        print(f"Train shape: {self.X_train.shape}, Test shape: {self.X_test.shape}")
        print(f"Train label distribution: {Counter(self.y_train)}")
        print(f"Test label distribution: {Counter(self.y_test)}")

    def get_sample_weights(self):
        # 클래스 불균형 보정을 위한 샘플 가중치 계산
        # balanced 옵션으로 각 클래스의 가중치를 자동 조정
        return compute_sample_weight(class_weight='balanced', y=self.y_train)

# 3단계: 평가 함수

In [3]:
def evaluate_model(name, y_true, y_pred):
    # 평가 결과 출력용 함수
    print(f"\n{name} Evaluation Results")
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print(classification_report(y_true, y_pred))

# 4단계: 모델 정의 클래스들

In [4]:
class DecisionTreeModel:
    def __init__(self):
        # 하이퍼파라미터 튜닝 결과 반영
        self.model = DecisionTreeClassifier(
            max_depth=5,
            min_samples_leaf=10,
            class_weight='balanced',  # 불균형 보정
            random_state=42
        )

    def train(self, X, y):
        self.model.fit(X, y)  # 모델 학습

    def predict(self, X):
        return self.model.predict(X)  # 예측값 반환


class RandomForestModel:
    def __init__(self):
        # 하이퍼파라미터 튜닝 결과 반영
        self.model = RandomForestClassifier(
            n_estimators=100,
            max_depth=7,
            class_weight='balanced',
            random_state=42
        )

    def train(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)


class GBDTModel:
    def __init__(self):
        # 하이퍼파라미터 튜닝 결과 반영
        self.model = GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=5,
            random_state=42
        )

    def train(self, X, y, sample_weights):
        # sample_weight 매개변수로 가중치 적용 가능
        self.model.fit(X, y, sample_weight=sample_weights)

    def predict(self, X):
        return self.model.predict(X)


class XGBoostModel:
    def __init__(self, scale_pos_weight):
        # scale_pos_weight로 불균형 조정
        self.model = xgb.XGBClassifier(
            n_estimators=100,
            learning_rate=0.05,
            max_depth=4,
            scale_pos_weight=scale_pos_weight,
            use_label_encoder=False,
            eval_metric='logloss',
            random_state=42
        )

    def train(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

#5단계: 파라미터 튜닝

In [5]:
# F1-score를 기반으로 모델 성능을 평가하기 위한 Scorer 생성
f1_scorer = make_scorer(f1_score)

# ---------------- Decision Tree 튜닝 ----------------
def tune_decision_tree(X, y):
    # 그리드 서치를 위한 파라미터 후보 목록 설정
    param_grid = {
        'max_depth': [3, 4, 5, 6, 7],              # 트리의 최대 깊이 후보
        'min_samples_leaf': [5, 10, 20]            # 리프 노드가 되기 위한 최소 샘플 수
    }

    # 클래스 불균형 문제를 고려하여 class_weight='balanced'로 설정
    dt = DecisionTreeClassifier(class_weight='balanced', random_state=42)

    # F1-score를 기준으로 교차검증을 통해 최적 파라미터 탐색
    grid = GridSearchCV(dt, param_grid, scoring=f1_scorer, cv=3, n_jobs=-1)

    # 모델 학습 (그리드 서치 수행)
    grid.fit(X, y)

    # 최적 파라미터 및 최고 F1-score 출력
    print("DecisionTree 최적 파라미터:", grid.best_params_)
    print("최고 F1 점수:", grid.best_score_)

    # 최적 모델 반환
    return grid.best_estimator_

# ---------------- Random Forest 튜닝 ----------------
def tune_random_forest(X, y):
    # 파라미터 후보 설정
    param_grid = {
        'n_estimators': [50, 100, 200],           # 트리 개수 후보
        'max_depth': [5, 7, 10]                   # 트리 최대 깊이 후보
    }

    # 클래스 불균형 처리 + 재현성 확보
    rf = RandomForestClassifier(class_weight='balanced', random_state=42)

    # F1-score 기준으로 그리드 서치 수행
    grid = GridSearchCV(rf, param_grid, scoring=f1_scorer, cv=3, n_jobs=-1)

    # 모델 학습
    grid.fit(X, y)

    # 최적 파라미터 및 성능 출력
    print("RandomForest 최적 파라미터:", grid.best_params_)
    print("최고 F1 점수:", grid.best_score_)

    # 최적 모델 반환
    return grid.best_estimator_

# ---------------- GBDT 튜닝 ----------------
def tune_gbdt(X, y):
    # 파라미터 후보 설정
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],        # 학습률 후보
        'max_depth': [3, 5, 7]                    # 트리 최대 깊이 후보
    }

    # GBDT 모델 설정 (기본 트리 수는 50)
    gbdt = GradientBoostingClassifier(n_estimators=50, random_state=42)

    # 그리드 서치 수행
    grid = GridSearchCV(gbdt, param_grid, scoring=f1_scorer, cv=3, n_jobs=-1)

    # 학습 및 탐색
    grid.fit(X, y)

    # 최적 파라미터 및 최고 성능 출력
    print("GBDT 최적 파라미터:", grid.best_params_)
    print("최고 F1 점수:", grid.best_score_)

    # 최적 모델 반환
    return grid.best_estimator_

# ---------------- XGBoost 튜닝 ----------------
def tune_xgboost(X, y, scale_pos_weight):
    # 파라미터 후보 설정
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],       # 학습률 후보
        'max_depth': [3, 4, 5]                    # 트리 최대 깊이 후보
    }

    # XGBoost 분류기 설정
    xgb_clf = xgb.XGBClassifier(
        n_estimators=50,                        # 트리 수
        scale_pos_weight=scale_pos_weight,      # 클래스 불균형 비율 설정
        use_label_encoder=False,                # 경고 제거
        eval_metric='logloss',                  # 평가 지표 설정
        random_state=42                         # 재현성 확보
    )

    # 그리드 서치를 통해 파라미터 최적화
    grid = GridSearchCV(xgb_clf, param_grid, scoring=f1_scorer, cv=3, n_jobs=-1)

    # 모델 학습
    grid.fit(X, y)

    # 최적 파라미터 및 F1 성능 출력
    print("XGBoost 최적 파라미터:", grid.best_params_)
    print("최고 F1 점수:", grid.best_score_)

    # 최적 모델 반환
    return grid.best_estimator_

# 6단계: 실행부

In [6]:
if __name__ == '__main__':
    loader = BinomialClassificationDataLoader(classification_data_path)
    loader.preprocess()

    print("\n--- Logistic Regression 평가 ---")
    logreg = LogisticRegression(penalty=None, class_weight='balanced', max_iter=1000, random_state=42)
    logreg.fit(loader.X_train, loader.y_train)
    y_pred_logreg = logreg.predict(loader.X_test)
    evaluate_model("Logistic Regression", loader.y_test, y_pred_logreg)

    print("\n--- Decision Tree 하이퍼파라미터 튜닝 및 평가 ---")
    best_dt = tune_decision_tree(loader.X_train, loader.y_train)
    y_pred_dt = best_dt.predict(loader.X_test)
    evaluate_model("Decision Tree_Tuned", loader.y_test, y_pred_dt)

    print("\n--- Random Forest 하이퍼파라미터 튜닝 및 평가 ---")
    best_rf = tune_random_forest(loader.X_train, loader.y_train)
    y_pred_rf = best_rf.predict(loader.X_test)
    evaluate_model("Random Forest_Tuned", loader.y_test, y_pred_rf)

    print("\n--- GBDT 하이퍼파라미터 튜닝 및 평가 ---")
    best_gbdt = tune_gbdt(loader.X_train, loader.y_train)
    y_pred_gbdt = best_gbdt.predict(loader.X_test)
    evaluate_model("Gradient Boosted Trees_Tuned", loader.y_test, y_pred_gbdt)

    print("\n--- XGBoost 하이퍼파라미터 튜닝 및 평가 ---")
    neg, pos = np.bincount(loader.y_train)
    scale_pos_weight = neg / pos
    best_xgb = tune_xgboost(loader.X_train, loader.y_train, scale_pos_weight)
    y_pred_xgb = best_xgb.predict(loader.X_test)
    evaluate_model("XGBoost_Tuned", loader.y_test, y_pred_xgb)


    #결과 분석 : LR 외 성능지표가 1에 수렴함. 데이터셋 불균형 해결, GridSearchCV를 통한 교차검증, train-test 데이터 구분 등 하였으나, 파라미터 튜닝 등 모델이 복잡할 가능성

Train shape: (58622, 70), Test shape: (14656, 70)
Train label distribution: Counter({np.int64(1): 29311, np.int64(0): 29311})
Test label distribution: Counter({np.int64(1): 7328, np.int64(0): 7328})

--- Logistic Regression 평가 ---

Logistic Regression Evaluation Results
Accuracy : 0.7765
Precision: 0.8669
Recall   : 0.6532
F1 Score : 0.7451
              precision    recall  f1-score   support

           0       0.72      0.90      0.80      7328
           1       0.87      0.65      0.75      7328

    accuracy                           0.78     14656
   macro avg       0.79      0.78      0.77     14656
weighted avg       0.79      0.78      0.77     14656


--- Decision Tree 하이퍼파라미터 튜닝 및 평가 ---
DecisionTree 최적 파라미터: {'max_depth': 3, 'min_samples_leaf': 5}
최고 F1 점수: 0.9999317778682048

Decision Tree_Tuned Evaluation Results
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1 Score : 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [14:51:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost 최적 파라미터: {'learning_rate': 0.01, 'max_depth': 3}
최고 F1 점수: 0.9999488360194423

XGBoost_Tuned Evaluation Results
Accuracy : 1.0000
Precision: 1.0000
Recall   : 1.0000
F1 Score : 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7328
           1       1.00      1.00      1.00      7328

    accuracy                           1.00     14656
   macro avg       1.00      1.00      1.00     14656
weighted avg       1.00      1.00      1.00     14656

